# Classifying Respiratory sounds using Deep Learning

## 7 Convolutional Neural Net for Wheeze and Crackle Sounds


In [85]:
import sys
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pydub

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### Model refinement

In our inital attempt, we were able to achieve a Classification Accuracy score of 60% accuracy on Wheeze and Crackle discrimination.  Using the CNN for it raises it ten points.

#### Feature Extraction refinement 

In the prevous feature extraction stage, the MFCC vectors would vary in size for the different audio files (depending on the samples duration). 

However, CNNs require a fixed size for all inputs. To overcome this we will zero pad the output vectors to make them all the same size. 

### Convolutional Neural Network (CNN) model architecture 


We will modify our model to be a Convolutional Neural Network (CNN) again using Keras and a Tensorflow backend. 

Again we will use a `sequential` model, starting with a simple model architecture, consisting of four `Conv2D` convolution layers, with our final output layer being a `dense` layer. 

The convolution layers are designed for feature detection. It works by sliding a filter window over the input and performing a matrix multiplication and storing the result in a feature map. This operation is known as a convolution. 


The `filter` parameter specifies the number of nodes in each layer. Each layer will increase in size from 16, 32, 64 to 128, while the `kernel_size` parameter specifies the size of the kernel window which in this case is 2 resulting in a 2x2 filter matrix. 

The first layer will receive the input shape of (40, 350, 1) where 40 is the number of MFCC's 350 is the number of frames taking padding into account and the 1 signifying that the audio is mono. 

The activation function we will be using for our convolutional layers is `ReLU` which is the same as our previous model. We will use a smaller `Dropout` value of 20% on our convolutional layers. 

Each convolutional layer has an associated pooling layer of `MaxPooling2D` type with the final convolutional layer having a `GlobalAveragePooling2D` type. The pooling layer is do reduce the dimensionality of the model (by reducing the parameters and subsquent computation requirements) which serves to shorten the training time and reduce overfitting. The Max Pooling type takes the maximum size for each window and the Global Average Pooling type takes the average which is suitable for feeding into our `dense` output layer.  

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is `softmax`. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [178]:
import numpy as np
max_pad_len = 350

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [95]:
import pandas as pd
import os
import librosa
import re
from scipy.io import wavfile
import math
from pydub import AudioSegment

cols = ['start', 'end','crackle','wheeze']
pattern = re.compile(r"^(.*)\.wav")

directory = './Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'
wavoutdir = "waves1"
outcsv = "sounds1.csv"
maxsize =40

# Set the path to the full UrbanSound dataset 
features = []

# Iterate through each sound file and extract the features 



fullwavpath = os.path.join(directory,wavoutdir)
if not os.path.exists(fullwavpath):
        os.makedirs(fullwavpath)
for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        fullpath = os.path.join(directory, filename)
        print(fullpath)
        result = pattern.match(filename)
        start = result.group(1)
        #print(start)
        annotationfile = directory+start+".txt"
        annot = pd.read_csv(annotationfile,names=cols,delimiter = "\t")
        myaudio = AudioSegment.from_file(fullpath, "wav") 
        print (len (myaudio))
        for i, row in annot.iterrows():
            wavfilename = start + str(i) +'.wav'
            outfile = os.path.join(fullwavpath , wavfilename)
            istart = math.floor(row['start']*1000)
            iend = math.floor(row['end']*1000)
            chunk_data = myaudio[istart:iend]
            chunk_data.export(outfile, format="wav")
            print(iend-istart)
            data = extract_features(outfile)
            if data is not None:
                class_label = ( 'both' if row['crackle'] and  row['wheeze']  
                               else 'crackle' if row['crackle']
                               else 'wheeze' if row['wheeze']
                               else 'none')
                if len(data)>0:
                    features.append([wavfilename,data, class_label,len(data)])
    
# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['filename','feature','class_label',"size"])
featuresdf.to_csv(outcsv)
print('Finished feature extraction from ', len(featuresdf), ' files') 

./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/157_1b1_Pl_sc_Meditron.wav
63100
3939
shape
170
5841
shape
252
3441
shape
149
4336
shape
187
5404
shape
233
6329
shape
273
4500
shape
194
5190
shape
224
4885
shape
211
4061
shape
175
4804
shape
207
5638
shape
243
3542
shape
153
./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/217_1b1_Tc_sc_Meditron.wav
20000
514
shape
23
2086
shape
90
2185
shape
95
2142
shape
93
2058
shape
89
2086
shape
90
2186
shape
95
2071
shape
90
2057
shape
89
2343
shape
101
200
shape
9
./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/205_4b2_Pr_mc_AKGC417L.wav
20000
3262
shape
141
3333
shape
144
3798
shape
164
3869
shape
167
3694
shape
160
./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/203_1p3_Ar_mc_AKGC417L.wav
20000
1857
shape
80
2131
shape
92
1905
shape
83
2000
shape
87
1953
shape
85
2095
shape
91
1952
shape
85
2072
shape
90
2178
shape
94
1726
shape
75
./

In [113]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [114]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 350
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows,num_columns,  num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows,num_columns,   num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows,num_columns , num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 


### Compiling the model 

For compiling our model, we will use the same three parameters as the previous model: 

In [115]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [116]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 39, 349, 16)       80        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 19, 174, 16)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 19, 174, 16)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 18, 173, 32)       2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 9, 86, 32)         0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 9, 86, 32)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 85, 64)        

### Training 

Here we will train the model. As training a CNN can take a sigificant amount of time, we will start with a low number of epochs and a low batch size. If we can see from the output that the model is converging, we will increase both numbers.  

In [117]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 5000
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 5507 samples, validate on 1377 samples
Epoch 1/10000
5507/5507 [==============================] - 11s 2ms/step - loss: 1.3232 - accuracy: 0.4868 - val_loss: 1.1482 - val_accuracy: 0.5352

Epoch 00001: val_loss improved from inf to 1.14823, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 2/10000
5507/5507 [==============================] - 11s 2ms/step - loss: 1.1232 - accuracy: 0.5170 - val_loss: 1.0989 - val_accuracy: 0.5592

Epoch 00002: val_loss improved from 1.14823 to 1.09894, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 3/10000
5507/5507 [==============================] - 11s 2ms/step - loss: 1.1041 - accuracy: 0.5237 - val_loss: 1.0767 - val_accuracy: 0.5664

Epoch 00003: val_loss improved from 1.09894 to 1.07674, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 4/10000
5507/5507 [==============================] - 11s 2ms/step - loss: 1.0982 - accuracy: 0.5239 - val_loss: 1.0803 - val_accuracy: 0.5730

Epoch 00004: val_loss

KeyboardInterrupt: 

### Test the model 

Here we will review the accuracy of the model on both the training and test data sets. 

In [118]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9974578022956848
Testing Accuracy:  0.7182280421257019


In [125]:
preds = model.predict(x_test)
preds

array([[1.3910585e-05, 2.2586063e-01, 7.6584953e-01, 8.2759121e-03],
       [9.6473994e-04, 1.2554581e-01, 8.6747020e-01, 6.0191741e-03],
       [4.4847042e-05, 9.1839921e-01, 8.1341140e-02, 2.1464446e-04],
       ...,
       [1.4643277e-03, 3.5322317e-01, 6.0281348e-01, 4.2498980e-02],
       [1.5376995e-03, 9.1883880e-01, 7.2584353e-02, 7.0391316e-03],
       [1.7330508e-03, 2.2583991e-01, 7.7084595e-01, 1.5811316e-03]],
      dtype=float32)

In [180]:
x_test.shape

(1377, 40, 350, 1)

In [126]:
y_test

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [145]:
len(featuresdf)

6884

In [147]:
len(y_test)

1377

In [148]:
len(y_train)

5507

In [143]:
outcome_list = []
for y, pred in zip(y_test,preds):
    alist = list(y)
    maxpos = alist.index(max(alist))  
    actual=le.inverse_transform([maxpos])[0]
    predlist = list(pred)
    maxpred = predlist.index(max(predlist))
    predicted = le.inverse_transform([maxpred])[0]
    outcome_list.append([actual,predicted]) 
cols = ["actual","predicted"]
outcomedf = pd.DataFrame(outcome_list,columns=cols)

In [198]:
labels = [le.inverse_transform([0])[0],
          le.inverse_transform([1])[0],
          le.inverse_transform([2])[0],
          le.inverse_transform([3])[0]]
labels

['both', 'crackle', 'none', 'wheeze']

In [144]:
outcomedf

,actual,predicted
0,none,none
1,none,none
2,none,crackle
3,wheeze,both
4,crackle,crackle
...,...,...
1372,wheeze,wheeze
1373,none,wheeze
1374,crackle,none
1375,crackle,crackle


In [171]:
outcomedf.groupby('actual')['predicted'].value_counts()

actual   predicted
both     both          22
         crackle       22
         wheeze        21
         none          18
crackle  crackle      258
         none         104
         both           9
         wheeze         4
none     none         628
         crackle      106
         wheeze        29
         both           4
wheeze   wheeze        81
         none          47
         crackle       13
         both          11
Name: predicted, dtype: int64

In [174]:
from sklearn.metrics import confusion_matrix
crackledf = outcomedf.replace('both','crackle')
crackledf= crackledf.replace('wheeze','none')
cm_crackle=confusion_matrix(crackledf['actual'].tolist(),crackledf['predicted'].tolist(),labels=["crackle","none"])
tp_crackle=cm_crackle[0,0]
tn_crackle=cm_crackle[1,1]
fn_crackle=cm_crackle[1,0]
fp_crackle=cm_crackle[0,1]
p_crackle = tp_crackle+fp_crackle
n_crackle = tn_crackle+fn_crackle
all_crackle = p_crackle+n_crackle
acc_crackle = (tp_crackle+tn_crackle)/all_crackle
sn_crackle=tp_crackle/p_crackle
sp_crackle=tn_crackle/n_crackle
print ("crackle accuracy,sensitivity, specificity:")
print (acc_crackle,sn_crackle,sp_crackle)
cm_crackle

crackle accuracy,sensitivity, specificity:
0.7959331880900509 0.6790393013100436 0.8541893362350381


array([[311, 147],
       [134, 785]])

In [172]:
from sklearn.metrics import confusion_matrix
wheezedf = outcomedf.replace('both','wheeze')
wheezedf= wheezedf.replace('crackle','none')
cm_wheeze=confusion_matrix(wheezedf['actual'].tolist(),wheezedf['predicted'].tolist(),labels=["wheeze","none"])
tp_wheeze=cm_wheeze[0,0]
tn_wheeze=cm_wheeze[1,1]
fn_wheeze=cm_wheeze[1,0]
fp_wheeze=cm_wheeze[0,1]
p_wheeze = tp_wheeze+fp_wheeze
n_wheeze = tn_wheeze+fn_wheeze
all_wheeze = p_wheeze+n_wheeze
acc_wheeze = (tp_wheeze+tn_wheeze)/all_wheeze
sn_wheeze=tp_wheeze/p_wheeze
sp_wheeze=tn_wheeze/n_wheeze
print ("wheeze accuracy,sensitivity, specificity:")
print (acc_wheeze,sn_wheeze,sp_wheeze)
cm_wheeze

wheeze accuracy,sensitivity, specificity:
0.8939724037763254 0.574468085106383 0.9597197898423818


array([[ 135,  100],
       [  46, 1096]])

The Training and Testing accuracy scores are both high and an increase on our initial model. Training accuracy has increased by ~20 pointsand Testing accuracy has increased by ~10 points.  It seems to be overfit.



### Predictions  

Here we will modify our previous method for testing the models predictions on a specified audio .wav file. 

In [119]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

### Validation 

#### Test with sample data 

As before we will verify the predictions using a subsection of the sample audio files we explored in the first notebook. We expect the bulk of these to be classified correctly. 

In [120]:
# Class: None

filename = './Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/waves1/118_1b1_Al_sc_Litt32000.wav' 
print_prediction(filename) 

shape
41
The predicted class is: none 

both 		 :  0.00010095169272972270846366882324
crackle 		 :  0.00146106304600834846496582031250
none 		 :  0.98539364337921142578125000000000
wheeze 		 :  0.01304431352764368057250976562500


In [121]:
# Class: Wheeze

filename = './Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/waves1/118_1b1_Al_sc_Litt32001.wav' 
print_prediction(filename) 

shape
51
The predicted class is: wheeze 

both 		 :  0.00000058522562085272511467337608
crackle 		 :  0.00000575412650505313649773597717
none 		 :  0.00208908203057944774627685546875
wheeze 		 :  0.99790453910827636718750000000000


In [122]:
# Class: crackle

filename = './Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/waves1/186_3b3_Pl_mc_AKGC417L3.wav'
print_prediction(filename) 

shape
150
The predicted class is: crackle 

both 		 :  0.00023268839868251234292984008789
crackle 		 :  0.97004055976867675781250000000000
none 		 :  0.02970980294048786163330078125000
wheeze 		 :  0.00001705625800241250544786453247


In [123]:
# Class: both

filename = './Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/waves1/130_2p5_Al_mc_AKGC417L10.wav' 
print_prediction(filename) 

shape
26
The predicted class is: both 

both 		 :  0.54650712013244628906250000000000
crackle 		 :  0.25836038589477539062500000000000
none 		 :  0.15726676583290100097656250000000
wheeze 		 :  0.03786578774452209472656250000000


#### Looping through a breath sample
We run the wheeze or crackle detector through entire breath samples.  These could use some that the data was trained on but ideally they would be new samples.  We choose one with a wheeze.  We create a tuple, that gives the wheeze value, and the crackle value, of a sample. This should be put above a threshold to decide whether there is a wheeze or crackle.  That threshold depends on the sensitivity and specificity of the tests.  The tests are highly specific,but insensitive. A value of 100.0 means that the entire sample is purely the sound.   

In [177]:
import pandas as pd
cols = ['start', 'end','crackle','wheeze']
df = pd.read_csv("./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/118_1b1_Al_sc_Litt3200.txt",
                 names=cols,delimiter = "\t")
df.head(25)

,start,end,crackle,wheeze
0,0.00000,0.93331,0,0
1,0.93331,2.10620,0,1
2,2.10620,3.37390,0,1
3,3.37390,4.44700,0,1
4,4.44700,5.55990,0,1
5,5.55990,6.75280,0,1
6,6.75280,7.94560,0,1
7,7.94560,9.06860,0,1
8,9.06860,15.47200,0,0


In [221]:
from pydub import AudioSegment

def crackle_wheeze_wav_alert(fullpath, sampling_size,labels):
    
    num_rows = 40
    num_columns = 350
    num_channels = 1
    #use jump if entire sample takes too long
    #jump = 1000
    namestart = fullpath[:-4] 
    myaudio = AudioSegment.from_file(fullpath, "wav") 
    millisecs = len (myaudio)
    laststart = millisecs-sampling_size-1
    breathlist =[]
    for i in range (laststart):
        #i = ii+jump
        wavfilename = namestart + str(i) +'.wav'
        end = i+sampling_size
        if end< len(myaudio):
            chunk_data = myaudio[i:end]
            chunk_data.export(wavfilename, format="wav")
            data = extract_features(wavfilename)
            shaped = data.reshape(num_rows,num_columns,num_channels)
            breathlist.append(shaped)
    breatharray= np.array(breathlist)
    predictlists = list(model.predict(breatharray))
    preds = [labels[list(predictlist).index(max(list(predictlist)))] for predictlist in predictlists]
    both= preds.count("both")
    none= preds.count("none")
    wheeze = preds.count("wheeze")
    crackle = preds.count("crackle")
    wheeze_val= ((wheeze+both)/len(preds))*100
    crackle_val = ((crackle+both)/len(preds))*100
    wheeze_crackle = (wheeze_val,crackle_val)
    
    return wheeze_crackle

In [222]:

sampling_size = 350
preds=crackle_wheeze_wav_alert("./Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/118_1b1_Al_sc_Litt3200.wav",
                                sampling_size,labels)
preds

(2.3080484094967266, 0.7935983069902783)

#### Observations 

We can see that the model performs 10 points better than the previous one, and can be used to detect a sound in a sample using a threshold

